<a href="https://colab.research.google.com/github/ASi-F/NLPlay-with-Transformers/blob/main/T5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers

In [2]:
import nltk
import pandas as pd

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/NLPlay/articles.xlsx')

In [5]:
df['Articles'] = df['Articles'].apply(lambda x: x.replace('\n',''))

In [6]:
df['length'] = df['Articles'].apply(len)

In [7]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

In [8]:
pip install SentencePiece

In [9]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('t5-base')

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

In [13]:
df['ref'] = df['Articles'].apply(lambda x: tokenizer.decode(tokenizer.encode(x[:2000],return_tensors='pt')[0],skip_special_tokens= True))

Token indices sequence length is longer than the specified maximum sequence length for this model (557 > 512). Running this sequence through the model will result in indexing errors


In [14]:
def gen(x):
  input_ids = tokenizer.encode(x[:100],return_tensors='pt')
  #output = model.generate(input_ids, max_length = 2000, num_beams = 2,no_repeat_ngram_size  = 2,early_stopping = True)
  output = model.generate(input_ids, max_length = 1000, do_sample = True, top_k = 100, top_p = 0.9)
  return output

df['gen'] = df['ref'].apply(gen)

In [35]:
from nltk.translate.bleu_score import SmoothingFunction, corpus_bleu, sentence_bleu


def bleu(ref, gen):
    ''' 
    calculate pair wise bleu score. uses nltk implementation
    Args:
        references : a list of reference sentences 
        candidates : a list of candidate(generated) sentences
    Returns:
        bleu score(float)
    '''
    gen = ref[:100].split()+gen
    len1 = len(ref)
    len2 = len(gen)
    if(len1 >= len2):
      ref = ref.split()[:len2]
    else:
      gen = gen[:len1]
      ref = ref.split()
    
    cc = SmoothingFunction()
    score_bleu = corpus_bleu([ref], [gen],  smoothing_function=cc.method7)
    return score_bleu

In [36]:
x = [bleu(df['ref'][i],df['gen'][i]) for i in range(len(df))]

In [37]:
dict_df = {'Scientific': x[0:10], 'Finance': x[10:20], 'Entertainment': x[20:30]}
BLEU_df = pd.DataFrame(dict_df)
BLEU_df.to_csv(r'BLEU_Scores_T5.csv', index=False)

In [38]:
df

,Articles,length,ref,gen
0,The dominant sequence transduction models are ...,2106,The dominant sequence transduction models are ...,"[mainly, involve, discrete, signal, transducti..."
1,The goal of reducing sequential computation al...,2190,The goal of reducing sequential computation al...,"[reducing, sequential, computation, also, form..."
2,We introduce a new language representation mod...,2176,We introduce a new language representation mod...,"[a, new, language, representation, model, call..."
3,"In this paper, we improve the fine-tuning base...",2049,"In this paper, we improve the fine-tuning base...","[algorithms, using, a, microtuning, approach.-..."
4,Many modern NLP systems rely on word embedding...,2597,Many modern NLP systems rely on word embedding...,"[most, modern, NLP, systems,, based, on, word,..."
5,We propose the Gaussian Error Linear Unit (GEL...,3425,We propose the Gaussian Error Linear Unit (GEL...,"[We, also, proposed, the, Gaussian, Error, Lin..."
6,This work presents a novel objective function ...,2165,This work presents a novel objective function ...,"[we, present, a, new, objective, function, for..."
7,Semantic Textual Similarity (STS) measures the...,2564,Semantic Textual Similarity (STS) measures the...,"[:, (STS), measures, the, meaning, similarity,..."
8,We present our UWB system for Semantic Textual...,2574,We present our UWB system for Semantic Textual...,"[We, present, our, system, for, Semantic, Text..."
9,The recent tremendous success of unsupervised ...,2414,The recent tremendous success of unsupervised ...,"[and, not, supervised, verb, embeddings, unsup..."


In [39]:
BLEU_df

,Scientific,Finance,Entertainment
0,0.000000,0.336966,0.325407
1,0.343101,0.346890,0.327977
2,0.324837,0.000000,0.338917
3,0.333886,0.325407,0.000000
4,0.332269,0.329371,0.000000
5,0.325961,0.335426,0.341850
6,0.336910,0.000000,0.334091
7,0.000000,0.000000,0.341583
8,0.000000,0.344943,0.338300
9,0.328988,0.000000,0.000000
